<a href="https://colab.research.google.com/github/ctkerns/cap4630_spring2020/blob/master/hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 5
## 1. General Concepts
Coming into this class, I already had a conceptual understanding of artificial intelligence, neural networks, and machine learning. Other than that, the material in this course was new to me.

* **Artificial Intelligence**: A field of computer science that uses computers to simulate intelligent behavior.
* **Machine Learning**: A category of Artificial Intelligence that uses machine learning algorithms to create a useful AI model, without a human explicitly creating the model.
* **Neural Network**: A model that is used in machine learning that uses connected "neurons" in layers to generate an output from an input.
* **Deep Learning**: A category of Machine Learning that uses neural networks with more layers to allow for a more complex model.

## 2. Basic Concepts
There are three types of machine learning: supervised, unsupervised, and reinforcement.
* **Unsupervised learning** is when correlations or categorizations of data is learned, but the data is unlabeled. We didn't learn too much about this type of learning in this class.
* **Supervised learning** creates a model that can predict labels using labeled training data. This was the type we focused on the most.
* **Reinforcement learning** is when a network continuously recieves new data through their environment. The network makes an action and uses the reward recieved based on its action to learn.  

There are also two types of machine learning models: classification and regression.
* **Classification** learns how data is grouped into categories. The output of this model is discrete.
* **Regression** learns how data affects certain values. The output of this model is continuous.  

There are two types of regression model learned in this course: linear regression and logistic regression.  
In **linear regression**, output is derived from input linearly. Every input is multiplied by a weight and added to a bias.  

$\hat{y} = x_1w_1+ x_2w_2 +\space...\space+ x_nw_n + b$  

Machine learning models are usually based on a linear regression model but apply a function to the output called an **activation function**. In the case of **logistic regression**, the activation is type of logistic function called a sigmoid function.

$\sigma(x) = \frac{1}{1\space+\space e^{-x}}$  

While a logistic regression model outputs continuous values, it is often used for binary classification problems by splitting the output range of the final sigma function into two parts divided by the midpoint.  

This process of using a model to calculate input values from output is called **forward propogation**.

* * *

In machine learning, a well designed model alone will not predict data well. It requires **training**, which is the process of minimizing the loss. **Loss** is calculated based on the difference between the output of the model when fed training data and the expected output. The **Loss function** is completed by applying a function to this difference. This is always necessary because a nonnegative result is required. Squaring the difference is commonly used.  

For a given data point, the weights of the model are an independent factor which influence the loss as a dependent factor. This can be visualized as a surface where loss is the vertical axis. Minimizing the loss can be done using a technique called **gradient descent**.

A **gradient** is like a derivative but it applies to a surface with multiple dependent variables. Like a derivative, it gives magnitude of a functions instantaneous increase, but it also provides the direction the function is increasing towards. Together this magnitude and direction makes a **gradient vector** at a point.  

The position of this gradient vector, and the current status of the network can be represented by a position vector where each component is a weight. Subtracting the gradient vector moves the position in the direction of the gradient vector. The magnitude can be adjusted by instead subtracting the normalized gradient vector multiplied by the **step size**, a **hyperparameter** which influences the **learning rate**. The process of changing weights in such a way is called **backpropogation**.

## 3. Building a Model

Libraries like Keras take care of implementing everything from section 2. When building a model, all the programmer has to do is provide the structure for the model by segmenting neural network layers together. A **layer** uses the same forward propogation calculations from section 2, but the output values become the inputs for the next layer. When several layers are stacked like this the network is said to be deeper. Training such a model is called **deep learning**.

In [0]:
import tensorflow as tf
from tensorflow import Keras

Creating a model in Keras takes just one line.

In [0]:
model = keras.models.Sequential()

Adding a layer is also very simple. For a linear regression model, the programmer only needs to specify the width of the layer in units.

In [0]:
layer = keras.layers.Dense(units=1)
model.add(layer)

Keras supports a wide range of activation functions. For a logistic regression model, simply change the activation function to sigmoid.

In [0]:
logistic_regression_layer = keras.layers.Dense(units=1, activation='sigmoid')

### Convolutional Neural Networks
**Convolutional Neural Networks**, also known as convnets or CNNs, are useful when the order of input variables matters. Computer vision is a common example. In computer vision, a network learns to recognize objects in an image. However, an object could be found in multiple locations in the image. Using the traditional rigid neural network structure would not work well considering this. For any visual feature, the network would have to learn how to recognize that feature for every location in the network. It has no ability to generalize this data correlation accross every adjacent combination of inputs.  

CNNs allow this ability to generalize correlations across adjacent combinations of data. They do this by storing features in **kernel matrices**. The input variables are represented by a matrix and are multiplied by the kernel matrix at various points. This multiplication is not regular matrix multiplication, it is akin to vector dot products. The kernel matrix moves over the input matrix, moving in each direction by a distance called **stride**. The stored output values from multiplication are called a feature map because they identify where in the image the feature is located. This computation is done in Keras by a **convolution layer**.  

CNNs also use pooling layers, typically added after every convolution layer. A **pooling layer** reduces the size of the feature map by moving a window over the feature map in the same way that the kernel matrix moves. In max pooling, the window takes the largest value and outputs it to a new feature map. The result consolidates redundant information and allows for a simpler classifier with less input.

After a number of convolutions and poolings, the resulting feature map is flattened (reduced to one dimension) and passed to the classifier. The **classifier** is a traditional neural network that uses the feature map to make a classification.

In Keras, it looks like this:

Create the network and add a convolutional layer with one filter and a kernel of size 3x3

In [0]:
model = keras.models.Sequential()
conv_layer = keras.layers.Conv2D(1, kernel_size = (3, 3))
model.add(conv_layer)

Add a pooling layer with pool size of 2x2 (default)

In [0]:
pool_layer = keras.layers.MaxPooling2D()
model.add(pool_layer)

Continue this process for desired number of convolution and pooling pairs, then add a classifier with 256 nodes in the first layer, and one binary classifier node at the end.

In [0]:
model.add(keras.layers.Flatten())
classifier_layer1 = keras.layers.Dense(256, activation='relu')
classifier_layer2 = keras.layers.Dense(1, activation='softmax')

## 4. Compiling a Model
Once a model has been built in Keras, it must be compiled. When compiling a network in Keras, you should specify the optimizer and the loss function.

### Optimizers
The optimizer is the part of the algorithm that optimizes the weights in a network to minimize loss. The optimizers in Keras are either variations of gradient descent or are different from gradient descent entirely.  

**Stochastic Gradient Descent** is one variation that randomly chooses a data point from the training data to train on. The intention of doing so is to limit the amount of computation needed when the training data is too large and redundant. The tradeoff is that the optimizer will take longer to converge on optimal accuracy because it will take a less direct route. This is because every data point has some noise and relying solely on one data point will skew the algorithm in a different direction because of this noise.  

**Mini-Batch Stochastic Gradient Descent** is a middle ground between SGD and traditional Gradient Descent. It neither processes one data point per iteration nor processes the entire set. Instead, it randomly assembles a smaller batch from the full data. This can be used to avoid the pitfalls of both optimizers.  

Choosing the optimizer algorithm is only one way to change the optimizer for better results. Certain hyperparameters can be changed as well, most notably the learning rate. Remember that at every iteration the vector representing the weights will change in the direction of the gradient vector by the distance of the learning rate. With a constant learning rate, the optimizer will never converge on the minimum loss. This is sometimes acceptable when the learning rate is small, and the divergence still closely fits the optimum point. The downside to this is that it will take a long time for the optimizer to near convergence. But, with a larger rate, the optimizer may diverge loosely around the optimum point. The solution is to reduce the learning rate over time.

The rmsprop optimizer included with Keras makes use of both comprimizes by using mini batches and dividing the learning rate as it converges.  

### Loss functions
Squared error loss is one option where the difference in outputs is squared and divided by two, which is the integral of the difference.

$L(y) = \frac{1}2 (y - \hat{y})^2$  

Binary cross entropy loss is more useful in binary classification problems.  

$L(y) = -y\cdot log(\hat y) - (1 - y)\cdot log(1 - \hat y)$  

### In Keras
Compiling a model in keras is as follows using rmsprop optimizer and binary cross entropy loss.


In [0]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics = ['accuracy'])

## 5. Training a Model
In keras, training can be done by specifying training input and output, batch size, and number of epochs.

In [0]:
model.fit(training_x, training_y, batch_size=20, epochs=30)

An **epoch** is the number of times the training algorithm trains on the entire data set.

Once training has completed, you will either have a network that can accurately predict separate test data, or one that can't. When training fails, the change in approach may not be clear, but it isn't total guesswork. By using **validation data** and monitoring **validation accuracy** you can detect whether your network is overfitting.  

**Overfitting** is when a network fails to make generalizations when learning and instead learns patterns that are specific to the training data. This can happen with small data sets or when the model is more complex than it needs to be. By reducing complexity, you take the network's ability to understand complicated patterns and force it to make generalizations to minimize loss.  

Adding a dropout layer can also prevent overfitting. A dropout layer randomly multiplies outputs of the previous layer to zero. This disrupts the network's learning and forces it to learn patterns in multiple ways which requires generalization.

**Underfitting** is when a network fails to minimize loss. If this happens after several epochs, it indicates that a model is inflexible. Networks are not magic. A network with one layer will never manage a task like computer vision. In this case, you can build up a network, but you should do it carefully. The structure of the network still needs to represent an accurate model of the problem with unknown parameters.



## 6. Finetuning a Pretrained Model
When it comes to CNNs, sometimes the low level feature recognition layers are applicable to other problems. In computer vision, researchers have spent years trying to find the best ways to configure a model to suit the task. They have also been training their models on very large datasets. It is unlikely you will do a better job than they do, so in most cases, it is best to start with a pretrained model and finetune it to your situation.  

Unless you find a model that is classifying the same objects that you are, it is unhelpful to reuse the classifier, because the classifier has been trained to recognize specific objects. Instead you should only use the convolutional base. In Keras, you can borrow Xceptions convolutional base like this:



In [0]:
from keras.applications import Xception

base = Xception(include_top = False, weights = 'imagenet', input_shape = (100, 100))

model = keras.models.Sequential()
model.add(bse)

By setting the include_top flag to false, we specify we don't want the classifier included. We set the weights to be the pretrained ones. In this case, the model will recognize 100x100 single channel images.  

Next you would construct the classifier. We can reuse the classifier from section 3.  

Then you can unfreeze layers to allow for some flexibility in the model. When unfreezing layers, it always makes sense to unfreeze starting from the top and continuing down continuously. For general purposes, it doesn't make much sense to unfreeze lower layers because these layers recognize very general features that are applicable to every classification problem. Upper layers may recognize features that are specific to certain objects, so it might be good to retrain these layers.  

Unfreezing a layer and not unfreezing the layers above it is inadviseable in any case. When you unfreeze that layer, it learns new features, but the layers above are trained to recognize the features from before making them incongruent and their output worthless.  

Keeping that in mind, a layer can be frozen like this in keras:


In [0]:
layer = model.get_layer('Xception').get_layer('block14_sepconv1')
layer.trainable = False

It might be a good idea to check the trainable status for every layer in the network before training as training takes a long time, and a simple mistake discovered late could waste that time.